# Autonomous driving car

## YOLO algorithm based on Research papers: 

```
Redmon et al., 2016

Redmon and Farhadi 2016
```

In [1]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.misc
import numpy as np
import pandas as pd

import PIL
from PIL import ImageFont, ImageDraw, Image

import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor

from keras.models import load_model

from yad2k.models.keras_yolo import yolo_head
from yad2k.utils.utils import draw_boxes, get_colors_for_classes, scale_boxes, read_classes, read_anchors, preprocess_image



2023-08-24 12:38:40.518303: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-24 12:38:40.519857: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-24 12:38:40.554791: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-24 12:38:40.555434: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 12:38:41.190816: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

### Anchor Boxes

Anchor boxes are chosen by exploring the training data to choose reasonable height/width ratios that represent the difference classes.

For this we have chosen 5 anchor boxes. almost 80 clases

(m, n_h, n_w, anchors, classes)

We use 19x19 size grids

In [2]:
# GRADED FUNCTION: yolo_filter_boxes

def yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = 0.6):
   '''
    box_confidence - tensor of shape (19, 19, 5, 1)- P_c confidence prob, for each 5 boxes predicted in each of the 19x19 cells
    boxes - (19, 19, 5, 4) -coordinates and dim of each 5 boxes in each cell, b_x, b_y, b_h, b_w
    box_class_probs - (19, 19, 5, 80) -class probabilities, c1, c2... c80 for each 80 classes for each of the 5 boxes per cells
   '''
        
   box_scores = box_class_probs*box_confidence
    
   box_classes = tf.math.argmax(box_scores,axis=-1)
   box_class_scores = tf.math.reduce_max(box_scores,axis=-1)
    
   filtering_mask = (box_class_scores >= threshold)
   
   scores = tf.boolean_mask(box_class_scores,filtering_mask)
   boxes = tf.boolean_mask(boxes,filtering_mask)
   classes = tf.boolean_mask(box_classes,filtering_mask)
   
   '''
    [TENSORs]    None - we can't predict that, it is based on the threshold
    scores - class prob for selected boxes (None, )
    boxes - (b_x, b_y, b_h, b_w) coordinates of boxes (None, 4)
    classes - index of the class detected by the selected boxes (None, )  
   ''' 
   return scores, boxes, classes

In [3]:
tf.random.set_seed(10)
box_confidence = tf.random.normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1)
boxes = tf.random.normal([19, 19, 5, 4], mean=1, stddev=4, seed = 1)
box_class_probs = tf.random.normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1)
scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = 0.5)
print("scores[2] = " + str(scores[2].numpy()))
print("boxes[2] = " + str(boxes[2].numpy()))
print("classes[2] = " + str(classes[2].numpy()))
print("scores.shape = " + str(scores.shape))
print("boxes.shape = " + str(boxes.shape))
print("classes.shape = " + str(classes.shape))

scores[2] = 9.270486
boxes[2] = [ 4.6399336  3.2303846  4.431282  -2.202031 ]
classes[2] = 8
scores.shape = (1789,)
boxes.shape = (1789, 4)
classes.shape = (1789,)


2023-08-24 12:38:41.836301: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-08-24 12:38:41.836341: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: mr-freak
2023-08-24 12:38:41.836347: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: mr-freak
2023-08-24 12:38:41.836446: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.86.5
2023-08-24 12:38:41.836463: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.86.5
2023-08-24 12:38:41.836469: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.86.5


### Non-Max Suppression (NMS)


In [4]:
def iou(box1, box2):
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1  # Bounding box coordinates (x1, y1, x2, y2)
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    
    # Coordinates of the intersection of two boxes
    xi1 = np.maximum(box1[0], box2[0])  # Upper left x-coordinate of intersection
    yi1 = np.maximum(box1[1], box2[1])  # Upper left y-coordinate of intersection
    
    xi2 = np.minimum(box1[2], box2[2])  # Lower right x-coordinate of intersection
    yi2 = np.minimum(box1[3], box2[3])  # Lower right y-coordinate of intersection
    
    # Calculate the dimensions of the intersection
    inter_width = xi2 - xi1
    inter_height = yi2 - yi1
    
    # Calculate the area of the intersection (intersection of non-negative dimensions)
    inter_area = max(inter_width, 0) * max(inter_height, 0)
    
    # Calculate the areas of both bounding boxes
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    # Calculate the area of the union by subtracting the intersection area from the sum of both box areas
    union_area = box1_area + box2_area - inter_area # by the formual, AUB = A + B - A (intersection) B
    
    # Calculate the Intersection over Union (IoU)
    iou = float(inter_area) / float(union_area)
    
    return iou


In [5]:
## YOLO non-max suppression
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    ''' [ output of yoloy_filter_boxes ]
    scores - tensot, (None, )
    boxes - (None, 1), that have been scaled to the image size
    classes - (None, ) 
    '''
    max_boxes_tensor = tf.Variable(max_boxes, dtype='int32')
    
    # to get the list of indices corresponding to boxes you keep
    nnm_indices = tf.image.non_max_suppression(boxes=boxes, 
                                               scores=scores, 
                                               max_output_size=max_boxes, 
                                               iou_threshold=iou_threshold)
    
    # Select only nnm_indices from scores, boxes and classes
    scores = tf.gather(scores,nnm_indices)
    
    boxes = tf.gather(boxes,nnm_indices)
    
    classes = tf.gather(classes,nnm_indices)
    
    return scores, boxes, classes


In [6]:
def yolo_boxes_to_corners(box_xy, box_wh):
    """Convert YOLO box predictions to bounding box corners."""
    box_mins = box_xy - (box_wh / 2.)
    box_maxes = box_xy + (box_wh / 2.)

    return tf.keras.backend.concatenate([
        box_mins[..., 1:2],  # y_min
        box_mins[..., 0:1],  # x_min
        box_maxes[..., 1:2],  # y_max
        box_maxes[..., 0:1]  # x_max
    ])


In [7]:
def yolo_eval(yolo_outputs, image_shape = (720, 1280), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    
    box_xy, box_wh, box_confidence, box_class_probs = yolo_outputs

    # Convert boxes to be ready for filtering functions (convert boxes box_xy and box_wh to corner coordinates)
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, score_threshold)
    
    # Scale boxes back to original image shape (720, 1280 )
    boxes = scale_boxes(boxes, image_shape) # YOLO Network was trained to run on 608x608 images

    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)

    return scores, boxes, classes

In [8]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
model_image_size = (608, 608) # Same as yolo_model input layer size

In [9]:
yolo_model = load_model("model_data/", compile=False)

In [10]:
yolo_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 608, 608, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 608, 608, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 608, 608, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 608, 608, 32)         0         ['batch_normalizati

In [11]:
len(yolo_model.layers)

75

In [12]:
def predict(image_file):
    """
    Runs the graph to predict boxes for "image_file". Prints and plots the predictions.
    
    Arguments:
    image_file -- name of an image stored in the "images" folder.
    
    Returns:
    out_scores -- tensor of shape (None, ), scores of the predicted boxes
    out_boxes -- tensor of shape (None, 4), coordinates of the predicted boxes
    out_classes -- tensor of shape (None, ), class index of the predicted boxes
    
    Note: "None" actually represents the number of predicted boxes, it varies between 0 and max_boxes. 
    """

    # Preprocess your image
    image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))
    
    yolo_model_outputs = yolo_model(image_data)
    yolo_outputs = yolo_head(yolo_model_outputs, anchors, len(class_names))
    
    out_scores, out_boxes, out_classes = yolo_eval(yolo_outputs, [image.size[1],  image.size[0]], 10, 0.3, 0.5)

    # Print predictions info
    print('Found {} boxes for {}'.format(len(out_boxes), "images/" + image_file))
    # Generate colors for drawing bounding boxes.
    colors = get_colors_for_classes(len(class_names))
    # Draw bounding boxes on the image file
    #draw_boxes2(image, out_scores, out_boxes, out_classes, class_names, colors, image_shape)
    draw_boxes(image, out_boxes, out_classes, class_names, out_scores)
    # Save the predicted bounding box on the image
    image.save(os.path.join("out", image_file), quality=100)
    # Display the results in the notebook
    output_image = Image.open(os.path.join("out", image_file))
    imshow(output_image)

    return out_scores, out_boxes, out_classes